### Tutorial covering using Miniai for Image Classification

This tutorial covers how to use the miniai library for image classification.  It begins with Mnist and then moves to the Imagenet Tiny.  The evolution of the models includes the use of data augmentation techniques.  Starting with Fashion Mnist

### Import Libraies and setup

In [ ]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl,numpy as np,matplotlib.pyplot as plt
import fastcore.all as fc
from collections.abc import Mapping
from pathlib import Path
from operator import attrgetter,itemgetter
from functools import partial
from copy import copy
from contextlib import contextmanager

import torchvision.transforms.functional as TF,torch.nn.functional as F
from torch import tensor,nn,optim
from torch.utils.data import DataLoader,default_collate
from torch.nn import init
from torch.optim import lr_scheduler
from torcheval.metrics import MulticlassAccuracy
from datasets import load_dataset,load_dataset_builder

In [ ]:
from miniai.datasets import DataLoaders, get_dls, collate_dict, inplace
from miniai.learner import Learner, TrainLearner, MetricsCB, DeviceCB, ProgressCB, LRFinderCB
from miniai.plotting import show_image, show_images
from miniai.model_blocks import conv, ResBlock, lin, pre_conv
from miniai.utils import set_seed, def_device
from miniai.activations import append_stats, ActivationStatsCB
from miniai.layers import GeneralRelu
from miniai.init import init_weights

NameError: name 'Callback' is not defined

### Set basic parameters and defaults

In [ ]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

import logging
logging.disable(logging.WARNING)

set_seed(42)

if fc.defaults.cpus>8: fc.defaults.cpus=8

### Load and configure Mnist Data 

In [ ]:
xl,yl = 'image','label'
name = "fashion_mnist"
bs = 1024
xmean,xstd = 0.28, 0.35

@inplace
def transformi(b): b[xl] = [(TF.to_tensor(o)-xmean)/xstd for o in b[xl]]

dsd = load_dataset(name)
tds = dsd.with_transform(transformi)
dls = DataLoaders.from_dd(tds, bs, num_workers=fc.defaults.cpus)

### Prepare Model callbacks, initialization, progress monitoring and metrics

Load the callback to capture activations during training.  In this case the module filter is set to GeneralRelu, which means that activations will be captured for every instance of that in the model. As an alternative the "mods" parameter can be used to define a list of layers for which activations will be captured.  This might be more appropriate for larger models to prevent activations from too many layers being captured.

In [ ]:
astats = ActivationStatsCB(module_filter=fc.risinstance(GeneralRelu))

Create a callback fo capture metrics during training.  In this case Multiclass accuracy.  Additional metrics can be created and assigned using keyword arguments.  Any metric defined using TorchEval should work.  In addition the metrics could be supplied in a list (without the keyword argument)

In [ ]:
metrics = MetricsCB(accuracy=MulticlassAccuracy())

In [ ]:
device_cb = DeviceCB()

Add a progress callback to print out and, optionally, plot training results

In [ ]:
progrss_cb = ProgressCB(plot=True)

Create a list of the callbacks to be used in the model.  Others can be added if needed

In [ ]:
cbs = [device_cb, metrics, progrss_cb, astats]

Create a activation function based on the GeneralRelu with predefined values for leak and sub

In [ ]:
act_gr = partial(GeneralRelu, leak=0.1, sub=0.4)

Define an initialisation function.  This will be applied to any conv layer and will assign a Kaiming normal initialisation using the value for leaky define

In [ ]:
iw = partial(init_weights, leaky=0.1)

Create a function to generate a ResNet style model using ResBlocks.  The ResBlocks have a conv block containing two conv layers, with the first conv block changing the number of channels and the second the resolution (if required).  Activations is by default only applied to the output form the first conv block.

As well as the conv block the ResBlock also has a pass through path that uses a pooling layer (if the resolution needs to be changed) and a conv layer (if the nuber of channels needs to change). After the outputs from the pass through path and the conv block are combined an optional activation can be applied

In the function below blocks are stacked with progressively increasing numbers of channels (if the defaults are used).  By default BatchNorm2d is used for normalisation, 

In [ ]:
def get_model(act=nn.ReLU, nfs=(8, 16, 32, 64, 128, 256), norm=nn.BatchNorm2d):
    layers = [ResBlock(1, nfs[0], stride=1, act=act, norm=norm)]
    layers += [ResBlock(nfs[i], nfs[i+1], stride=2, act=act, norm=norm) for i in range(len(nfs)-1)]
    layers += [nn.Flatten(), nn.Linear(nfs[-1], 10, bias=False), nn.BatchNorm1d(10)]
    return nn.Sequential(*layers).to(def_device)

In [ ]:
model = get_model(act=act_gr, norm=nn.BatchNorm2d).apply(iw)

Create a learner using the callbacks and model created

In [ ]:
learn = Learner(model=model, dls=dls, loss_func=F.cross_entropy, lr=0.001, cbs=cbs)

In [ ]:
learn.summary()